In [ ]:
import math
import numpy as np
from scipy.stats import norm
from scipy import interpolate
import cvxpy as cp

### 14.2.4 Applying Breden-Litzenberger in Practice

The following piece of code can be used to implement the Breeden-Litzenberger technique in Python:

In [ ]:
def callPx(s_0, k, r, sigma, tau):
	sigmaRtT = (sigma * math.sqrt(tau))
	rSigTerm = (r + sigma * sigma/2.0) * tau
	d1 = (math.log(s_0/k) + rSigTerm) / sigmaRtT
	d2 = d1 - sigmaRtT
	term1 = s_0 * norm.cdf(d1)
	term2 = k * math.exp(-r * tau) * norm.cdf(d2)
	price = term1 - term2
	return  price

def breeden_litzenberger(strikes, vols, s_0, rf, tau, dk):

    # use linear interpolation of implied volatility
    vol_interp = interpolate.interp1d(strikes, vols, fill_value="extrapolate")

    # loop through arrays and use Breeden-Litzenberger to find Risk Neutral Density at each point
    phis = np.full(len(strikes), 0.00)
    for zz, strike_zz in enumerate(strikes):
        px_up = callPx(s_0, strike_zz + dk, rf, vol_interp(strike_zz + dk), tau)
        px = callPx(s_0, strike_zz, rf, vol_interp(strike_zz), tau)
        px_dn = callPx(s_0, strike_zz - dk, rf, vol_interp(strike_zz - dk), tau)

        numer = (px_up - 2 * px + px_dn)
        denom = (dk * dk)
        phis[zz] = numer / denom

    return phis


# example
s0 = 100.0
r = 0.0
exp_t = 0.25
upper_k = 150.0
lower_k = 50.0
n_points = 10001

dk = (upper_k - lower_k) / n_points

ks = np.linspace(lower_k, upper_k, n_points)
ivs = np.full(n_points, 0.2)

rnd = breeden_litzenberger(ks, ivs, s0, r, exp_t, dk)
print(rnd)

[-1.42136970e-10  2.13205455e-10 -2.84273941e-10 ...  5.87113452e-06
  5.85470254e-06  5.83831477e-06]


### 14.3.2 Digital Options

Following 14.2.4, now we show how to price a digital option given a risk neutral PDF. Please make sure to run the above cell before executing the following one:

In [ ]:
def digi_call_px(k, rf, tau, nodes, probs, dk):
    digi_px = 0.0
    for zz, node_zz in enumerate(nodes):
        if node_zz >= k:
            digi_px += math.exp(-rf*tau) * probs[zz] * dk

    return digi_px

k = 100.0     # strike price
digital_call_price = digi_call_px(k, r, exp_t, ks, rnd, dk)
print(digital_call_price)

0.48019218977361167


### 14.5.5 Implementation of Weighted Monte Carlo in Practice: S&P Options

The following coding example shows a simple implementation of Weighted Monte Carlo, where the optimization is done on a set of terminal probabilities, and where no prior distribution is incorporated. As we reuse function `callPx()`  defined in 14.2.4, please make sure to run the cell under 14.2.4 before executing the following one:


In [ ]:
def weighted_mc(call_strikes, impl_vols, s0, tau, rf, num_nodes, lower_k, upper_k):

    nodes = np.linspace(lower_k, upper_k, num_nodes)
    x = cp.Variable(shape=num_nodes)
    obj = cp.Maximize(cp.sum(cp.entr(x)))
    forward = s0 * math.exp(rf * tau)
    constraints = [
        cp.sum(x) == 1,
        cp.sum(cp.multiply(x, nodes)) == forward,
    ]
    threshold = s0 * 0.005

    for k, iv in zip(call_strikes, impl_vols):
        payoffs = np.exp(-rf * tau) * np.maximum(nodes - k, 0.0)
        call_price = callPx(s0, k, rf, iv, tau)
        exp_payoff = cp.sum(cp.multiply(x, payoffs))
        constraints += [
            exp_payoff <= call_price + threshold,
            exp_payoff >= call_price - threshold,
        ]

    prob = cp.Problem(obj, constraints)
    prob.solve(verbose=True)

    return x.value


# example
s0 = 100.0
r = 0.0
exp_t = 0.25
upper_k = 150.0
lower_k = 50.0
n_points = 501

ks = np.linspace(lower_k, upper_k, n_points)
ivs = np.full(n_points, 0.2)

wmc_probs = weighted_mc(ks, ivs, s0, exp_t, r, n_points, lower_k, upper_k)
print(wmc_probs)

(CVXPY) Dec 27 07:34:22 PM: Your problem has 501 variables, 1004 constraints, and 0 parameters.
(CVXPY) Dec 27 07:34:22 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 27 07:34:22 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 27 07:34:22 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 27 07:34:22 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Dec 27 07:34:22 PM: Compiling problem (target solver=CLARABEL).
(CVXPY) Dec 27 07:34:22 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> CLARABEL
(CVXPY) Dec 27 07:34:22 PM: Applying reduction FlipObjective
(CVXPY) Dec 27 07:34:22 PM: Applying reduction Dcp2Cone
(CVXPY) Dec 27 07:34:22 PM: Applying reduction CvxAttr2Constr
(CVXPY) Dec 27 07:34:22 PM: Applying reduction ConeMatrixStuffing


                                     CVXPY                                     
                                     v1.6.7                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------


(CVXPY) Dec 27 07:34:24 PM: Applying reduction CLARABEL
(CVXPY) Dec 27 07:34:24 PM: Finished problem compilation (took 2.077e+00 seconds).
(CVXPY) Dec 27 07:34:24 PM: Invoking solver CLARABEL  to obtain a solution.


-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
-------------------------------------------------------------
           Clarabel.rs v0.11.1  -  Clever Acronym                

                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 1002
  constraints   = 2507
  nnz(P)        = 0
  nnz(A)        = 252504
  cones (total) = 503
    :        Zero = 1,  numel = 2
    : Nonnegative = 1,  numel = 1002
    : Exponential = 501,  numel = (3,3,3,3,...,3)

settings:
  linear algebra: direct / faer, precision: 64 bit (2 threads)
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-8, tol_gap_abs = 1.0e-8, tol_gap_rel = 1.0e-8,
  static

(CVXPY) Dec 27 07:34:27 PM: Problem status: optimal
(CVXPY) Dec 27 07:34:27 PM: Optimal value: 5.481e+00
(CVXPY) Dec 27 07:34:27 PM: Compilation took 2.077e+00 seconds
(CVXPY) Dec 27 07:34:27 PM: Solver (including time spent in interface) took 3.162e+00 seconds


 27  -5.4808e+00  -5.4808e+00  1.12e-08  1.27e-10  1.17e-11  3.14e-11  1.82e-11  6.34e-01  
 28  -5.4808e+00  -5.4808e+00  2.41e-09  2.75e-11  2.53e-12  6.79e-12  3.94e-12  7.92e-01  
---------------------------------------------------------------------------------------------
Terminated with status = Solved
solve time = 3.116330372s
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
[2.04182280e-05 2.09767290e-05 2.15505066e-05 2.21399789e-05
 2.27455751e-05 2.33677360e-05 2.40069151e-05 2.46635775e-05
 2.53382016e-05 2.60312791e-05 2.67433142e-05 2.74748254e-05
 2.82263453e-05 2.89984229e-05 2.97916184e-05 3.06065100e-05
 3.14436919e-05 3.23037728e-05 3.31873801e-05 3.40951562e-05
 3.50277655e-05 3.59858790e-05 3.69702031e-05 3.79814512e-05
 3.90203600e-05 4.00876860e-05 4.11842067e-05 4.23107208e-